In [39]:
import pandas as pd
import numpy as np
from math import ceil

In [40]:
payment_api_export = pd.read_csv('C:/SAG/Account Management Workflow/files/Payment_api_Nov_11.csv', low_memory=False)
payment_mixpanel_export = pd.read_csv('C:/SAG/Account Management Workflow/files/Payment_mixpanel_Nov_11.csv')
pipedrive_contacts = pd.read_csv('C:/SAG/Account Management Workflow/files/pipedrive_contacts_updated.csv', low_memory=False)
unpaid_user = pd.read_csv('C:/SAG/Account Management Workflow/files/unpaid_user_signup.csv')
print(len(payment_api_export))
print(len(payment_mixpanel_export))
print(len(pipedrive_contacts))
payment_api_export.head(2)

279521
6854
345899


,event,time,distinct_id,$distinct_id_before_identity,$email,$insert_id,$mp_api_endpoint,$mp_api_timestamp_ms,3DS,Amount,...,Billing ZIP,Card First Name,Card Last Name,$is_reshuffled,Fraud Reason,ID at Stripe,mp_lib,Discount Line Items,$preshuffle_distinct_id,Billing Email
0,New Payment Made,1759577430,$device:18f1d12ba0e79b-0070bdfcabb2b9-26001d51...,raymondrod2500@gmail.com,raymondrod2500@gmail.com,ojCxFauwdiDppFFjhDjxztkBwsbrwaCtxpmz,api.mixpanel.com,1759577430178,False,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,New Payment Made,1759586025,$device:18a46e20df2447-059e9f5d132d1-1a525634-...,aseverini@genesis-global.com,aseverini@genesis-global.com,gBmsdvCnyEznpczgterDnapzCothvfCuauBv,api.mixpanel.com,1759586025819,False,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
pay = payment_api_export[['time', '$email', 'distinct_id', '$distinct_id_before_identity']]
pay.head(2)

,time,$email,distinct_id,$distinct_id_before_identity
0,1759577430,raymondrod2500@gmail.com,$device:18f1d12ba0e79b-0070bdfcabb2b9-26001d51...,raymondrod2500@gmail.com
1,1759586025,aseverini@genesis-global.com,$device:18a46e20df2447-059e9f5d132d1-1a525634-...,aseverini@genesis-global.com


In [42]:
pay['$email'].isnull().sum()

np.int64(1654)

In [43]:
pay = pay.copy()
pay["email"] = pay.apply(
    lambda x: (
        x["$email"]
        if "@" in str(x["$email"])
        else (
            x["$distinct_id_before_identity"]
            if "@" in str(x["$distinct_id_before_identity"])
            else (
                x["distinct_id"]
                if "@" in str(x["distinct_id"])
                else None
            )
        )
    ),
    axis=1
)
pay['email'].isnull().sum()

np.int64(1653)

In [44]:
pay = pay.copy()

pay.columns = pay.columns.str.strip().str.title()
pay["Time"] = pd.to_datetime(pay["Time"], unit="s")
pay["Date"] = pay["Time"].dt.date

pay = pay[['Email', 'Time', 'Date']]
pay.head(2)

,Email,Time,Date
0,raymondrod2500@gmail.com,2025-10-04 11:30:30,2025-10-04
1,aseverini@genesis-global.com,2025-10-04 13:53:45,2025-10-04


In [45]:
pay = pay.dropna(subset=['Email'])
print(len(pay))
pay['Email'].isnull().sum()

277868


np.int64(0)

In [46]:
# Group by Email to get first and last payment dates
pay1 = pay.groupby('Email')['Date'].agg(
    First_Payment='min',
    Last_Payment='max'
).reset_index()
pay1.head(2)

,Email,First_Payment,Last_Payment
0,0.0@hrops.co.uk,2025-04-21,2025-04-21
1,000311avas@gmail.com,2024-01-17,2024-01-17


In [47]:
# Group and transform to match original DataFrame shape
pay['First_Payment'] = pay.groupby('Email')['Date'].transform('min')
pay['Last_Payment'] = pay.groupby('Email')['Date'].transform('max')
pay.head(2)

,Email,Time,Date,First_Payment,Last_Payment
0,raymondrod2500@gmail.com,2025-10-04 11:30:30,2025-10-04,2024-04-28,2025-11-07
1,aseverini@genesis-global.com,2025-10-04 13:53:45,2025-10-04,2023-08-30,2025-11-04


We can calculate the difference in months as:

Months = (current year − first year ) × 12 + (current month − first month) + ((current day − first day​)/30)

In [48]:
today = pd.Timestamp.today()

pay1['First_Payment'] = pd.to_datetime(pay1['First_Payment'])

def months_since_first(date):
    delta_years = today.year - date.year
    delta_months = today.month - date.month
    delta_days = today.day - date.day
    total_months = delta_years * 12 + delta_months + delta_days / 30
    return ceil(total_months)

pay1['Duration_Months'] = pay1['First_Payment'].apply(months_since_first)
pay1 = pay1.sort_values(by = 'Email', ascending=True)
pay1.head(2)

,Email,First_Payment,Last_Payment,Duration_Months
0,0.0@hrops.co.uk,2025-04-21,2025-04-21,7
1,000311avas@gmail.com,2024-01-17,2024-01-17,22


In [49]:
pay2 = payment_mixpanel_export.copy()
pay2.head(2)

,Email,Phone Number,Phone Number Country,Workspace,A. Payment (all time),B. Amount (Year),C. Amount (Month)
0,01mihir.54@gmail.com,undefined,undefined,Agrifolio,40.0,0.0,0.0
1,1032484261@qq.com,undefined,undefined,RXCOOLERS INTERNATIONAL LIMITED,98.0,98.0,15.0


In [50]:
pay2 = pay2.groupby('Email').agg({
    'A. Payment (all time)': 'sum',
    'B. Amount (Year)': 'sum',
    'C. Amount (Month)': 'sum',
    'Workspace': 'first'
}).reset_index()
print(len(pay2))
pay2.tail(5)

6297


,Email,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace
6292,zulujimmy71@gmail.com,19.00,19.00,19.0,Jimmy
6293,zuomio2025@gmail.com,45.00,45.00,15.0,ZUOMIO LLC
6294,zuzanalabelle@gmail.com,533.68,334.98,25.5,Antonio cesare Sanvido
6295,zz+krispcall@primalgs.com,15.00,15.00,15.0,Zoltan
6296,zzomega@gmail.com,105.00,105.00,15.0,Devon


In [51]:
pay_merged = pd.merge(pay1, pay2, on = 'Email', how = 'right')
print(len(pay_merged))
pay_merged.head(2)

6297


,Email,First_Payment,Last_Payment,Duration_Months,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace
0,01mihir.54@gmail.com,2025-11-03,2025-11-03,1.0,40.0,0.0,0.0,Agrifolio
1,1032484261@qq.com,2025-05-13,2025-10-13,7.0,98.0,98.0,15.0,RXCOOLERS INTERNATIONAL LIMITED


In [52]:

pay_merged['Amount_per_month'] = np.where(
    pay_merged['Duration_Months'] > 0,
    pay_merged['A. Payment (all time)'] / pay_merged['Duration_Months'],
    pay_merged['A. Payment (all time)']  # if Duration_Months is 0
)

pay_merged['Amount_per_month'] = pay_merged['Amount_per_month'].round(2)
pay_merged.head(2)


,Email,First_Payment,Last_Payment,Duration_Months,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace,Amount_per_month
0,01mihir.54@gmail.com,2025-11-03,2025-11-03,1.0,40.0,0.0,0.0,Agrifolio,40.0
1,1032484261@qq.com,2025-05-13,2025-10-13,7.0,98.0,98.0,15.0,RXCOOLERS INTERNATIONAL LIMITED,14.0


In [53]:

def assign_tier(row):
    duration = row['Duration_Months']
    amount = row['Amount_per_month']
    
    # VIP
    if duration >= 24 and amount >= 30:
        return 'VIP'
    
    # Platinum
    if (duration >= 12 and amount > 120) or \
       (duration >= 6 and amount > 180) or \
       (duration >= 3 and amount > 300):
        return 'Platinum'
    
    # Gold
    if (duration >= 6 and amount > 80) or \
       (duration >= 3 and amount > 120):
        return 'Gold'
    
    # Silver
    if (duration >= 6 and amount >= 60) or \
       (duration >= 3 and amount > 80):
        return 'Silver'
    
    # Bronze
    return 'Bronze'

pay_merged['Tier'] = pay_merged.apply(assign_tier, axis=1)
pay_merged.head(2)


,Email,First_Payment,Last_Payment,Duration_Months,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace,Amount_per_month,Tier
0,01mihir.54@gmail.com,2025-11-03,2025-11-03,1.0,40.0,0.0,0.0,Agrifolio,40.0,Bronze
1,1032484261@qq.com,2025-05-13,2025-10-13,7.0,98.0,98.0,15.0,RXCOOLERS INTERNATIONAL LIMITED,14.0,Bronze


In [54]:
pipedrive_contacts.head(2)

,email,phone,phone_country_name,first_name,last_name,full_name
0,prabhatkumar33200@gmail.com,+918521225200,India,Prabhat Kumar,Singh,Prabhat Kumar Singh
1,stetoc37@gmail.com,+48723717522,Poland,Stefan,Toczek,Stefan Toczek


In [55]:
pipedrive_contacts.columns = pipedrive_contacts.columns.str.title()
pipedrive_contacts.columns

Index(['Email', 'Phone', 'Phone_Country_Name', 'First_Name', 'Last_Name',
       'Full_Name'],
      dtype='object')

In [56]:
pipedrive_contacts['Email'] = pipedrive_contacts['Email'].str.strip()
pipedrive_contacts['Phone_Country_Name'] = pipedrive_contacts['Phone_Country_Name'].str.strip()

In [57]:
print(pipedrive_contacts.isnull().sum())
print(pipedrive_contacts.duplicated().sum())
print(pipedrive_contacts['Email'].duplicated().sum())

Email                     0
Phone                     0
Phone_Country_Name     4150
First_Name            26157
Last_Name             10638
Full_Name                 1
dtype: int64
2323
10052


In [58]:
pipedrive_contacts = pipedrive_contacts.drop_duplicates()

In [59]:
empty_country = pipedrive_contacts['Phone_Country_Name'].isnull()
pipedrive_contacts[empty_country]


,Email,Phone,Phone_Country_Name,First_Name,Last_Name,Full_Name
48,abc@xyz.com,+11234567890,NaN,spez,NaN,spez
51,abc@xyz.com,+13984763829,NaN,Test,Run,Test Run
52,abc@qwe.com,+10987324,NaN,Test,Run,Test Run
523,rijadihalimi75@gmail.com,+38348988888,NaN,rijad,halimi,rijad halimi
1799,albtrip123@gmail.com,+38349995277,NaN,Alb,Trip,Alb Trip
...,...,...,...,...,...,...
343795,meriton-gashi7@hotmail.com,+38344177522,NaN,Meriton,Gashi,Meriton Gashi
344621,rajendra+100@krispcall.com,+11111111111,NaN,DineshHQ,NaN,DineshHQ
344672,npgfa2025@gmail.com,+18130692749,NaN,Neil,Reddy,Neil Reddy
344965,gezimbozhlani@proton.me,+38343882825,NaN,Gezim,NaN,Gezim


In [60]:
pipedrive_contacts = pipedrive_contacts.drop_duplicates(subset='Email')

In [61]:
pay_merged_contacts = pd.merge(pay_merged, pipedrive_contacts, on = 'Email', how = 'left')
print(len(pay_merged_contacts))
pay_merged_contacts.head(2)

6297


,Email,First_Payment,Last_Payment,Duration_Months,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace,Amount_per_month,Tier,Phone,Phone_Country_Name,First_Name,Last_Name,Full_Name
0,01mihir.54@gmail.com,2025-11-03,2025-11-03,1.0,40.0,0.0,0.0,Agrifolio,40.0,Bronze,+19047850458,United States,Daniel,Smith,Daniel Smith
1,1032484261@qq.com,2025-05-13,2025-10-13,7.0,98.0,98.0,15.0,RXCOOLERS INTERNATIONAL LIMITED,14.0,Bronze,+8617816877409,China,Niki,Lee,Niki Lee


In [62]:
pay_merged_contacts.drop_duplicates(subset = 'Email', inplace = True)
print(len(pay_merged_contacts))

6297


In [ ]:
unpaid_user.head(2)
unpaid_user = unpaid_user[['$email', 'Phone Number', 'Phone Number Country']]
unpaid_user = unpaid_user.rename(columns={'$email': 'Email'})

KeyError: "['Email'] not in index"

In [ ]:
print(len(unpaid_user))
print(unpaid_user.duplicated().sum())
print(unpaid_user['Email'].duplicated().sum())
print(unpaid_user.isnull().sum())

113
20
20
Email                   0
Phone Number            0
Phone Number Country    0
dtype: int64


In [ ]:
unpaid_user = unpaid_user.drop_duplicates(subset='Email')

In [ ]:
final_merged = pd.merge(pay_merged_contacts, unpaid_user, on = "Email", how = 'left')
print(len(final_merged))
final_merged.head(5)

6297


,Email,First_Payment,Last_Payment,Duration_Months,A. Payment (all time),B. Amount (Year),C. Amount (Month),Workspace,Amount_per_month,Tier,Phone,Phone_Country_Name,First_Name,Last_Name,Full_Name,Phone Number,Phone Number Country
0,01mihir.54@gmail.com,2025-11-03,2025-11-03,1.0,40.0,0.0,0.0,Agrifolio,40.00,Bronze,+19047850458,United States,Daniel,Smith,Daniel Smith,NaN,NaN
1,1032484261@qq.com,2025-05-13,2025-10-13,7.0,98.0,98.0,15.0,RXCOOLERS INTERNATIONAL LIMITED,14.00,Bronze,+8617816877409,China,Niki,Lee,Niki Lee,NaN,NaN
2,1088global@gmail.com,2025-09-28,2025-09-28,2.0,393.0,393.0,0.0,global,196.50,Bronze,+61493239960,Australia,global,glo,global glo,NaN,NaN
3,115408@mail.ee,2024-04-19,2025-10-19,19.0,300.0,195.0,15.0,Alex,15.79,Bronze,+79032108249,Russian Federation,Alex,Kud,Alex Kud,NaN,NaN
4,11viktor11myhal@gmail.com,2025-10-19,2025-10-19,1.0,15.0,15.0,15.0,Viktor,15.00,Bronze,+380987082275,Ukraine,Viktor,NaN,Viktor,NaN,NaN


In [ ]:
print(final_merged.duplicated().sum())
print(final_merged['Email'].duplicated().sum())
print(final_merged.isnull().sum())

0
0
Email                       0
First_Payment               1
Last_Payment                1
Duration_Months             1
A. Payment (all time)       0
B. Amount (Year)            0
C. Amount (Month)           0
Workspace                   1
Amount_per_month            0
Tier                        0
Phone                     119
Phone_Country_Name        171
First_Name                194
Last_Name                 668
Full_Name                 119
Phone Number             6297
Phone Number Country     6297
dtype: int64


In [ ]:
undefined_values = ['', ' ', '  ', 'undefined', 'Undefined', 'none', 'None', 'nan', 'NaN']

final_merged['Phone Number'] = final_merged['Phone Number'].astype(str).str.strip()
final_merged['Phone'] = final_merged['Phone'].astype(str).str.strip()
final_merged['Phone_Country_Name'] = final_merged['Phone_Country_Name'].astype(str).str.strip()
final_merged['Phone Number Country'] = final_merged['Phone Number Country'].astype(str).str.strip()

final_merged['Phone Number'] = final_merged['Phone Number'].replace(undefined_values, pd.NA)
final_merged['Phone'] = final_merged['Phone'].replace(undefined_values, pd.NA)
final_merged['Phone_Number'] = final_merged['Phone Number'].fillna(final_merged['Phone'])

final_merged['Phone_Country_Name'] = final_merged['Phone_Country_Name'].replace(undefined_values, pd.NA)
final_merged['Phone Number Country'] = final_merged['Phone Number Country'].replace(undefined_values, pd.NA)
final_merged['Phone_Country'] = final_merged['Phone_Country_Name'].fillna(final_merged['Phone Number Country'])


final_merged = final_merged[['Email', 'Full_Name', 'First_Name', 'Last_Name', 'Phone_Number', 'Phone_Country', 'First_Payment', 'Last_Payment', 
                             'Duration_Months', 'A. Payment (all time)', 'B. Amount (Year)', 'C. Amount (Month)', 'Workspace', 'Amount_per_month', 'Tier']]

final_merged.head(2)

NameError: name 'final_merged' is not defined

In [ ]:
tier_summary = final_merged.groupby('Tier').agg(
    Number_of_Users=('Email', 'count')
).reset_index()

# Calculate percentages
tier_summary['Percentage'] = (tier_summary['Number_of_Users'] / tier_summary['Number_of_Users'].sum() * 100).round(2)

# Optional: order tiers
tier_order = ['VIP', 'Platinum', 'Gold', 'Silver', 'Bronze']
tier_summary['Tier'] = pd.Categorical(tier_summary['Tier'], categories=tier_order, ordered=True)
tier_summary = tier_summary.sort_values('Tier').reset_index(drop=True)

tier_summary

NameError: name 'final_merged' is not defined